In [1]:
import xml.etree.ElementTree as ET, getopt, logging, sys, random, re, copy, os
import numpy as np
from collections import Counter
import operator
import sklearn
import nltk

from xml.sax.saxutils import escape
import xgboost
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import pandas as pd

D:\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [83]:
def prepare_df(fname):
    sentences_array =[]
    categories=[]
    ote=[]
    pol=[]
    reviews = ET.parse(fname).getroot().findall('Review')
    i = 1
    for review in reviews:
        sentences = review[0] #get the sentences
       
        for sentence in sentences:
            
            if (len(sentence) > 1):

                opinions = sentence[1] #getting the opinions field
                
                if ( len(opinions) > 0): #check if there are aspects 
                   
                    t = sentence[0].text #getting the text
                    cat=''
                    aspect=''
                    polarity = ''
                    for opinion in opinions: 
                        print sentence[0].text
                        if(opinion.attrib['target']):
                                    print opinion.attrib['target']
                                    cat = cat + ' '+ opinion.attrib['category'].lower()
                                    aspect = aspect + ';'+ opinion.attrib['target'].lower()
                                    if opinion.attrib['polarity'].lower() in ['positve','psoitive']:
                                            polarity = polarity + ' ' + 'positive'
                                    else:
                                        polarity = polarity + ' ' + opinion.attrib['polarity'].lower()
                    ote.append(aspect) #store the category
                    categories.append(cat)
                    pol.append(polarity)
                    sentences_array.append(sentence[0].text)
                
    return sentences_array,categories,ote,pol

In [127]:
sentences_array,categories,ote,polarity = prepare_df('revisedDatasets-teachers/revisedLabels/teacherReviews_500_bazla.xml')
data = pd.DataFrame({'text' : sentences_array,'category' : categories,'polarity':polarity,'aspect term' : ote})


One of  the most impressive lecture at Stanford.
lecture
Well-organized lecture and tough course work (programming assignment.)
lecture
Well-organized lecture and tough course work (programming assignment.)
course work
He's great
null
The best prof through whom I could learn online 
null
Great intro course for anyone who wants to get into programming, beats codeacademy and other sorts.
intro course
Great intro course for anyone who wants to get into programming, beats codeacademy and other sorts.
programming
He is simply a super awesome.
null
He is hilarious and knows how to teach the material he aimed to cover within the planned time which is extraordinary for me.   
material
He is hilarious and knows how to teach the material he aimed to cover within the planned time which is extraordinary for me.   
null
Mehran Sahami, what Stanford is all about, great teaching, new ideas, helping you ask big questions and build awesome stuff.
null
Mehran Sahami, what Stanford is all about, great te

In [128]:
data.shape

(1434, 4)

In [129]:
sentences_array,categories,ote,polarity = prepare_df('revisedDatasets-teachers/revisedLabels/teacherReviews_500_sumaira.xml')
data2 = pd.DataFrame({'text' : sentences_array,'category' : categories,'polarity':polarity,'aspect term' : ote})


Nice lady but has no business teaching Philosophy.
null
Nice lady but has no business teaching Philosophy.
Philosophy
Reads off a note card and seems to have trouble answering arbitrary but relavent questions considering her lecture material.
note card
Reads off a note card and seems to have trouble answering arbitrary but relavent questions considering her lecture material.
questions
Became hard to sit in class.
null
Pathetic is a better word for it.
null
I really liked this prof!!
prof
He was definately my favorite.
null
Very helpful and easily approachable.
null
Makes sure you know your stuff and prepares you for the final.
final
Makes you work hard, and gives hard tests but they aren't worth much.
tests
Great Professor.
Professor
Very lively lectures and amazingly approachable.
lectures
Very lively lectures and amazingly approachable.
null
He spent 1 hour with me trying to figure out what I wanted to major in.
null
Pushes chemistry a lot but obviously only because he loves it and t

In [130]:
data2.shape

(1486, 4)

In [131]:
sentences_array,categories,ote,polarity = prepare_df('revisedDatasets-teachers/revisedLabels/50ReviewsRateMyProfessor.xml')
data3 = pd.DataFrame({'text' : sentences_array,'category' : categories,'polarity':polarity,'aspect term' : ote})
data3.shape

Does a great job at explaining the material clearly, and her lectures are interesting and engaging.
lectures
Does a great job at explaining the material clearly, and her lectures are interesting and engaging.
material
If you want an A in this class you have to read the book and assigned articles, as well as prepare for quizzes, she is a hard grader so you won't get away with not knowing the material on the exams. 
hard grader
Dr. Hora is a talker, she does love it so.
null
Her teaching style did not resonate well with me.
teaching style
She gave an exam before a single quiz was given.
null
For this reason, I can not recommend Hora
null
She is rough.
null
basically don't skip and talk a lot and she will like you, but if you are a person who doesn't like talking in class, you're screwed.
null
She's very bias towards the Political Science students.
null
Everyone else should forget taking her.
null
The study guide she gives has nothing to do with the exam.
study guide
The book is boring an

(148, 4)

In [132]:
data = data.append(data2.append(data3))

In [133]:
data.shape

(3068, 4)

In [134]:
def unique_labels(df,colname):
    labels = []
    for i,row in df.iterrows():
        cat = row[str(colname)].split()
        for j in range(0,len(cat)):
            labels.append(cat[j])
            if cat[j] == 'positve':
                print row['text']
    res = pd.DataFrame({'col':labels})
    return res['col'].unique()

In [135]:
print unique_labels(data,'category')

['teaching' 'exam' 'teacher' 'relevant' 'answering' 'course' 'approachable'
 'attendance' 'knowledge' 'grading' 'class' 'late']


In [136]:
print unique_labels(data,'polarity')

['positive' 'negative' 'neutral']


In [137]:
from sklearn.model_selection import train_test_split


In [138]:
train, test = train_test_split(data, test_size = 0.2)

In [139]:
train.shape

(2454, 4)

In [140]:
test.shape

(614, 4)

In [141]:
train.to_csv('teacher_train_revised_labels.csv',sep='\t',index=False)

In [142]:
test.to_csv('teacher_test_revised_labels.csv',sep='\t',index=False)

In [100]:
tr = pd.read_csv('teacher_train_revised.csv',sep='\t')

In [101]:
tr.shape

(2454, 4)